In [20]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as numpy
import random
import pickle
from collections import Counter
import codecs

#nltk.download()
totalLinesToRead = 100
lemmatizer = WordNetLemmatizer()

In [26]:
def lexicon_init(positive, negative):
    lex = []
    for file1 in [positive, negative]:
        with open(file1, 'r') as fil:
            fil = codecs.open(file1, encoding='utf-8')
            contains = fil.readlines()
            for line in contains[:totalLinesToRead]:
                #print(line)
                words = word_tokenize(line)
                lex += list(words)
    lex = [lemmatizer.lemmatize(i) for i in lex]
    wordC = Counter(lex)
    finalLex = []
    for word in wordC:
        if 50 < wordC[word] < 1000:
            finalLex.append(word)
    print(len(finalLex))
    return finalLex
#text = 'Reese Witherspoon plays Dani, a young country girl that falls madly in love with the new 17 year old neighbor, Court, played by Jason London.'
#word_tokenize(text)

In [36]:
def samples(classif, lex, section):
    features = []
    with open(section, 'r') as file1:
        contents = file1.readlines()
        for line in contents[:totalLinesToRead]:
            words = word_tokenize(line.decode('utf-8').lower())
            words = [lemmatizer.lemmatize(i) for i in words]
            feats = numpy.zeros(len(lex))
            for w in words:
                if w.lower() in lex:
                    index = lex.index(w.lower())
                    feats[index] += 1
            feats = list(feats)
            features.append([feats, classif])
            
    return features

In [39]:
def create_labels_plus_sets(positive, negative, size=0.1):
    lex = lexicon_init(positive, negative)
    features = []
    #sample,lex,classif
    features += samples([1,0], lex, './data/netflix-train-pos')
    features += samples([0,1], lex, './data/netflix-train-neg')
    random.shuffle(features)
    
    features = numpy.array(features)
    testSize = int(len(features)*size)
    training_one = list(features[:,0][:-testSize])
    training_two = list(features[:,1][:-testSize])
    testing_one = list(features[:,0][-testSize:])
    testing_two = list(features[:,1][-testSize:])
    
    return training_one, training_two, testing_one, testing_two
    

In [40]:
#if __name__ == '__main__':
training_one, training_two, testing_one, testing_two = create_labels_plus_sets('./data/netflix-train-pos', './data/netflix-train-neg')
with open('sentiment_set.pickle', 'wb') as fil:
    pickle.dump([training_one, training_two, testing_one, testing_two], fil)
        

127


In [46]:
#import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

training_one, training_two, testing_one, testing_two = create_labels_plus_sets('./data/netflix-train-pos', './data/netflix-train-neg')

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

x = tf.placeholder('float', [None, len(training_one[0])])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(training_one[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            i = 0
            while i < len(training_one):
                start = i
                end = i+batch_size
                batch_one = numpy.array(training_one[start:end])
                batch_two = numpy.array(training_two[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_one, y: batch_two})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:testing_one, y:testing_two}))

train_neural_network(x)

127
('Epoch', 1, 'completed out of', 10, 'loss:', 84755.34765625)
('Epoch', 2, 'completed out of', 10, 'loss:', 47096.23828125)
('Epoch', 3, 'completed out of', 10, 'loss:', 52282.806640625)
('Epoch', 4, 'completed out of', 10, 'loss:', 49376.408203125)
('Epoch', 5, 'completed out of', 10, 'loss:', 35786.87890625)
('Epoch', 6, 'completed out of', 10, 'loss:', 27104.1201171875)
('Epoch', 7, 'completed out of', 10, 'loss:', 26614.53515625)
('Epoch', 8, 'completed out of', 10, 'loss:', 24232.240234375)
('Epoch', 9, 'completed out of', 10, 'loss:', 17243.171875)
('Epoch', 10, 'completed out of', 10, 'loss:', 11912.2265625)
('Accuracy:', 0.55000001)
